In [1]:
import os
import sys
sys.path.append('/aiffel')

import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam

from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [2]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1) # 마지막 hiddel layer의 차원 (임베딩 차원)
                                                        # 1개의 보상 점수 출력하는 선형화 layer
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [3]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

# with 문(context manager) : 
# - context 안에서 모델을 생성하면, 분산 학습이나 메모리 최적화 등의 전략이 적용된다

# NaiveStrategy : 분산 학습 전략 클래스로 NaiveStrategy는 아무 분산 전략도 적용하지 않는 기본 전략
with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  501219 KB |  501219 KB |  501219 KB |       0 B  |
|       from large pool |  488448 KB |  488448 KB |  488448 KB |       0 B  |
|       from small pool |   12771 KB |   12771 KB |   12771 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  501219 KB |  501219 KB |  501219 KB |       0 B  |
|       from large pool |  488448 KB |  488448 KB |  488448 KB |

In [5]:
with open('./data/clean_kochatgpt_1_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []

for tmp in list_data_dict:
    prompt = tmp['prompt']
    ranking = tmp['ranking']

    for index in range(1, len(ranking)):
        n = ranking[0]
        m = ranking[index]


        data = {
             'prompt': prompt,
             'chosen': tmp['completion_{}'.format(n)],
             'rejected': tmp['completion_{}'.format(m)]
        }

        total_data_ranking2chosen.append(data)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))

before data num: 10126
after  data num: 20252


In [ ]:
with open('./data/clean_kochatgpt_1_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []

for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)

    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))

In [6]:
print('data example: \n%s'%total_data_ranking2chosen[0])
print('data example: \n%s'%total_data_ranking2chosen[1])

data example: 
{'prompt': '번디는 자신이 탐정잡지 , 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나 ?', 'chosen': '라이언에게 말했다 .', 'rejected': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다 .'}
data example: 
{'prompt': '번디는 자신이 탐정잡지 , 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나 ?', 'chosen': '라이언에게 말했다 .', 'rejected': 'allow me to answer your question . i know that you are curious about me .'}


In [7]:
import torch.distributed as dist


def is_rank_0() -> bool:
    return not dist.is_initialized() or dist.get_rank() == 0


class RewardDataset(Dataset):
    """
    Dataset for reward model

    Args:
        dataset: dataset for reward model
        tokenizer: tokenizer for reward model
        max_length: max length of input
    """

    def __init__(self, dataset, tokenizer: Callable, max_length: int) -> None:
        super().__init__()
        self.chosen = []
        self.reject = []
        for data in tqdm(dataset, disable=not is_rank_0()):
            prompt = data['prompt']

            chosen = prompt + data['chosen'] + "<|endoftext|>"
            chosen_token = tokenizer(chosen,
                                     max_length=max_length,
                                     padding="max_length",
                                     truncation=True,
                                     return_tensors="pt")
            self.chosen.append({
                "input_ids": chosen_token['input_ids'],
                "attention_mask": chosen_token['attention_mask']
            })

            reject = prompt + data['rejected'] + "<|endoftext|>"
            reject_token = tokenizer(reject,
                                     max_length=max_length,
                                     padding="max_length",
                                     truncation=True,
                                     return_tensors="pt")
            self.reject.append({
                "input_ids": reject_token['input_ids'],
                "attention_mask": reject_token['attention_mask']
            })

    def __len__(self):
        length = len(self.chosen)
        return length

    def __getitem__(self, idx):
        return self.chosen[idx]["input_ids"], self.chosen[idx]["attention_mask"], self.reject[idx][
            "input_ids"], self.reject[idx]["attention_mask"]


In [10]:
total_len = len(total_data_ranking2chosen)

# train 80%, val 10%, test 10%로 나누기
train_size = int(0.8 * total_len)
val_size = int(0.1 * total_len)
test_size = total_len - train_size - val_size  # 남는 거 처리

train_data = total_data_ranking2chosen[:train_size] 
eval_data = total_data_ranking2chosen[train_size:train_size+val_size]
test_data = total_data_ranking2chosen[train_size+val_size:]

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)
test_dataset = RewardDataset(test_data, tokenizer, 512)

print(f"Train: {len(train_dataset)}")
print(f"Validation: {len(eval_dataset)}")
print(f"Test: {len(test_dataset)}")

100%|██████████| 2026/2026 [00:01<00:00, 1179.48it/s]

Train: 16201
Validation: 2025
Test: 2026


In [23]:
# RM 모델 평가
from tqdm import tqdm

def get_rm_score(model, input_text):
    input_ids = tokenizer.encode(prompt + input_text, return_tensors='pt').to(torch.cuda.current_device())

    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    return output_reward

def inference_RM(model, test_data) : 
    total = 0
    correct = 0
    for raw in tqdm( test_data, desc="Inferencing" )  : 
        prompt = raw['prompt']
    
        chosen = prompt + " " + raw['chosen'] + "<|endoftext|>"
        rejected = prompt + " " + raw['rejected'] + "<|endoftext|>"

    
        score_chosen = get_rm_score( model, chosen )
        score_rejected = get_rm_score( model, rejected )
    
        if score_chosen > score_rejected:
            correct += 1
        total += 1
    
        if total <= 2 :
            print('prompt : ', prompt)
            print('choson : %s\nreward score: %.1f'%(chosen, score_chosen))
            print('reject : %s\nreward score: %.1f'%(rejected, score_rejected))
            print()        
    
    accuracy = correct / total
    print(f"RM total: {total}")
    print(f"RM correct: {correct}")
    print(f"RM accuracy: {accuracy:.4f}")

In [24]:
# before training 
inference_RM( model, test_data)

Inferencing:   0%|          | 4/2026 [00:00<01:03, 31.70it/s]

prompt :  파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ?
choson : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? 나폴레옹은 프랑스에서 수십만의 신병을 징집했습니다 .<|endoftext|>
reward score: 0.9
reject : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? 나폴레옹은 프랑스에서 수십만의 신병을 징집했습니다 .<|endoftext|>
reward score: 0.9

prompt :  파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ?
choson : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? 나폴레옹은 프랑스에서 수십만의 신병을 징집했습니다 .<|endoftext|>
reward score: 0.9
reject : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? i was walking around and i saw a man on a horse . i don t know if i was walking or on a horse .<|endoftext|>
reward score: 0.9



Inferencing: 100%|██████████| 2026/2026 [00:53<00:00, 37.98it/s]

RM total: 2026
RM correct: 871
RM accuracy: 0.4299


In [25]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=3)

trainer.fit(use_lora=0)

model.save_pretrained('./models/output_2_RM')

Train step of epoch 0:   2%|▏         | 91/4051 [01:22<59:15,  1.11it/s, loss=0.339]


Train step of epoch 0:   4%|▍         | 182/4051 [02:45<58:21,  1.11it/s, loss=0.346]


Train step of epoch 0:   7%|▋         | 273/4051 [04:07<56:57,  1.11it/s, loss=0.385]


Train step of epoch 0:   9%|▉         | 364/4051 [05:29<55:39,  1.10it/s, loss=0.0328]


Train step of epoch 0:  11%|█         | 455/4051 [06:52<54:15,  1.10it/s, loss=0.00134]


...

Train step of epoch 0:  92%|█████████▏| 3717/4051 [55:49<05:01,  1.11it/s, loss=0.0119]


Train step of epoch 0:  94%|█████████▍| 3806/4051 [57:09<03:34,  1.14it/s, loss=7.81e-5]


Train step of epoch 0:  96%|█████████▌| 3895/4051 [58:29<02:20,  1.11it/s, loss=0.377]


Train step of epoch 0:  98%|█████████▊| 3984/4051 [59:49<00:59,  1.12it/s, loss=0.000515]


Train epoch: 100%|██████████| 1/1 [1:03:17<00:00, 3797.19s/it]


In [26]:
# after training 
inference_RM( model, test_data)

Inferencing:   0%|          | 4/2026 [00:00<01:04, 31.16it/s]

prompt :  파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ?
choson : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? 나폴레옹은 프랑스에서 수십만의 신병을 징집했습니다 .<|endoftext|>
reward score: -0.8
reject : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? 나폴레옹은 프랑스에서 수십만의 신병을 징집했습니다 .<|endoftext|>
reward score: -0.8

prompt :  파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ?
choson : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? 나폴레옹은 프랑스에서 수십만의 신병을 징집했습니다 .<|endoftext|>
reward score: -0.8
reject : 파리로 돌아온 나폴레옹은 어느 나라에서 수십만의 신병을 징집했습니까 ? i was walking around and i saw a man on a horse . i don t know if i was walking or on a horse .<|endoftext|>
reward score: -1.2



Inferencing: 100%|██████████| 2026/2026 [00:53<00:00, 37.80it/s]

RM total: 2026
RM correct: 1928
RM accuracy: 0.9516


# <회고>

### 실험 기록
- 1 epoch에 1시간정도 소요 (batch size =4 )
- epoch를 늘려서 해볼것 !!



### test summary 

✅ Accuracy 점수

|  |   total_count | correct 수 | accuracy |
| ---- | --- |  --- | --- |
| RM 전 | 2026 | 871 | 0.4299 |
| RM 후 | 2026 | 1928 | 0.9516 |
 
 - 1 epoch만 돌렸는데도 accuracy가 0.42 -> 0.95로 상승했다 
